In [1]:
import numpy as np
import pandas as pd
#import datetime as dt
import pymongo
import json
import datetime as dt
import pickle
import math
#import os
from bs4 import BeautifulSoup as bs
#import requests
from splinter import Browser
import time
import re
import datetime 
import dateutil.parser
import string

In [2]:
# create mongo db connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# connect to my nba data db
db = client.nba_data_db

In [3]:
# create splinter browser
def init_browser(noShowBrowser):
    executable_path = {'executable_path': 'chromedriver.exe'}
    browser = Browser('chrome', **executable_path, headless=noShowBrowser)
    return browser

In [4]:
browser = init_browser(False)

In [5]:
nba_odds_url = "https://www.actionnetwork.com/nba/live-odds"

In [6]:
browser.visit(nba_odds_url)

In [18]:
# parse html in soup
html = browser.html
# Parse with soup
soup = bs(html, 'html.parser')

In [19]:
# Find all tables
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))

In [20]:
df[0].columns = df[0].columns.droplevel()

In [21]:
df_odds = df[0][['SCHEDULEDSCHEDULED','OPEN']]

In [22]:
df_odds

,SCHEDULEDSCHEDULED,OPEN
0,523CHA524ORL4:00 PM,+3 -110-3 -110
1,525NYK526ATL4:30 PM,+6 -110-6 -110
2,527OKC528NOP5:00 PM,-4 -110+4 -110


In [156]:
gameDate = datetime.date.today().strftime("%x")
gameDate

'02/13/19'

In [133]:
gameDate = datetime.date.today().strftime("%x")
gameDate = datetime.date.today() - datetime.timedelta(1)
gameDate = gameDate.strftime("%x")

In [134]:
#temp = time.strptime(gameDate, '%m/%d/%y')
#temp.isoformat()
date = parser.parse(gameDate).isoformat()
date

'2019-02-12T00:00:00'

In [23]:
roadTeam = []
homeTeam = []
pointSpread = []
for index, row in df_odds.iterrows():
    game = row['SCHEDULEDSCHEDULED']
    roadTeam.append(game[3:6])
    homeTeam.append(game[9:12])
    pointSpread.append(-1.0*float(row['OPEN'].split()[0]))
    #print(row['OPEN'])

In [26]:
homeTeam

['ORL', 'ATL', 'NOP']

In [14]:
def getOddsDf():
    # parse html in soup
    html = browser.html
    # Parse with soup
    soup = bs(html, 'html.parser')
    # Find all tables
    table = soup.find_all('table')[0] 
    df = pd.read_html(str(table))
    df[0].columns = df[0].columns.droplevel()
    df_odds = df[0][['SCHEDULEDSCHEDULED','OPEN']]
    return df_odds

In [12]:
def getDailyOdds(df_odds):
    roadTeam = []
    homeTeam = []
    pointSpread = []
    gameDate = dateutil.parser.parse(datetime.date.today().strftime("%x")).isoformat()
    #gameDate = dateutil.parser.parse((datetime.date.today()- datetime.timedelta(1)).strftime("%x")).isoformat()
    for index, row in df_odds.iterrows():
        game = row['SCHEDULEDSCHEDULED']
        roadTeam.append(game[3:6])
        homeTeam.append(game[9:12])
        if(row['OPEN'].split()[0] == 'PK'):
            pointSpread.append(0.0)
        else:
            pointSpread.append(-1.0*float(row['OPEN'].split()[0]))
    oddsDict = {'GameDate' : gameDate,
                'roadTeam' : roadTeam,
                'homeTeam' : homeTeam,
                'pointSpread' : pointSpread
               }
    return oddsDict

In [28]:
def getDailyOddsPast(df_odds, daysAgo):
    roadTeam = []
    homeTeam = []
    pointSpread = []
    gameDate = dateutil.parser.parse((datetime.date.today()- datetime.timedelta(daysAgo)).strftime("%x")).isoformat()
    for index, row in df_odds.iterrows():
        game = row['SCHEDULEDSCHEDULED']
        roadTeam.append(game[3:6])
        if(game[9].isalpha()):
            homeTeam.append(game[9:12])
        elif(game[11].isalpha()):
            homeTeam.append(game[11:14])
        else:
            homeTeam.append(game[12:15])
        if(row['OPEN'].split()[0] == 'PK'):
            pointSpread.append(0.0)
        else:
            pointSpread.append(-1.0*float(row['OPEN'].split()[0]))
    oddsDict = {'GameDate' : gameDate,
                'roadTeam' : roadTeam,
                'homeTeam' : homeTeam,
                'pointSpread' : pointSpread
               }
    return oddsDict

In [141]:
temp = getDailyOdds(df_odds)

In [157]:
temp = getDailyOddsPast(df_odds, 1)

In [29]:
df_odds = getOddsDf()
temp = getDailyOddsPast(df_odds, 0)
temp

{'GameDate': '2019-02-14T00:00:00',
 'roadTeam': ['CHA', 'NYK', 'OKC'],
 'homeTeam': ['ORL', 'ATL', 'NOP'],
 'pointSpread': [-3.0, -6.0, 4.0]}

In [30]:
db.nba_odds.insert_one(temp)

In [110]:
def getPointsSpread():
    # create mongo db connection
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    # connect to my nba data db
    db = client.nba_data_db
    temp = db.nba_odds.find().sort('GameDate',pymongo.DESCENDING).limit(1)
    date_index = temp[0]['GameDate']
    last_date = dateutil.parser.parse(date_index)
    last_date = last_date.strftime('%m/%d/%Y')
    temp = list(temp)
    for i in temp:
        i.pop('_id', None)
    temp[0]['GameDate'] = last_date
    return temp[0]

In [111]:
dailyOdds = getPointsSpread()
dailyOdds

{'GameDate': '02/11/2019',
 'roadTeam': ['LAL', 'SAS', 'ORL', 'BOS', 'UTA'],
 'homeTeam': ['ATL', 'MEM', 'NOP', 'PHI', 'GSW'],
 'pointSpread': [5.5, 4.5, -4.5, -5.0, -9.5]}

In [35]:
temp = db.nba_team_current_elo.find()
temp = list(temp)
for i in temp:
    i.pop('_id', None)
df_elo = pd.DataFrame(temp).T.reset_index()

In [37]:
df_elo.columns = ['Team', 'ELO']

In [46]:
df_elo = df_elo.sort_values(['ELO'], ascending=False).reset_index(drop=True)

In [47]:
df_elo.to_json(orient='columns')

'{"Team":{"0":"GSW","1":"TOR","2":"MIL","3":"OKC","4":"DEN","5":"HOU","6":"PHI","7":"BOS","8":"IND","9":"UTA","10":"POR","11":"SAS","12":"LAC","13":"SAC","14":"MIN","15":"BKN","16":"MIA","17":"DET","18":"LAL","19":"NOP","20":"CHA","21":"ORL","22":"WAS","23":"DAL","24":"ATL","25":"MEM","26":"CLE","27":"CHI","28":"PHX","29":"NYK"},"ELO":{"0":1675.5920753664,"1":1670.3931310332,"2":1658.1458472072,"3":1621.9333169611,"4":1620.883200795,"5":1612.3141838139,"6":1609.6539417177,"7":1606.1994232384,"8":1596.9245586204,"9":1594.998126242,"10":1591.207661975,"11":1557.0098678961,"12":1514.1894740622,"13":1502.8510831605,"14":1491.5325571331,"15":1487.4396251909,"16":1472.1572658327,"17":1465.974613901,"18":1462.9035832076,"19":1461.7944236782,"20":1457.9319757768,"21":1451.6432644501,"22":1450.2733459055,"23":1444.8333268861,"24":1393.3587904009,"25":1374.4285153569,"26":1340.5690804595,"27":1315.3342114183,"28":1254.8896038372,"29":1242.639924476}}'